In [1]:
#hide
#default_exp docs

# nbprocess.docs
- Exporting a notebook to docs

In [19]:
#export
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.export import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.xtras import *

import uuid

In [3]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

In [12]:
#     res = ''
#     for cell in nb.cells:
#         src = cell.source
#         if cell.cell_type=='code': src = f"\n```{lang}\n{src}\n```\n"
#         res += src + '\n'
#         if 'outputs' in cell:
#             for outp in cell.outputs:
#                 d = outp.get('data', {})
#                 if 'text/html' in d:
#                     res += '\n'.join(d['text/html'])
#                 elif 'text/plain' in d:
#                     t = '\n'.join(d['text/plain'])
#                     res += f"```\n  {t}\n```\n"
#     return res

In [4]:
def rm_blank_proc(cell):
    if(cell.source.strip()==''): cell.source = None

In [20]:
#export
_re_ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

def strip_ansi_proc(cell):
    "Strip Ansi Characters"
    for o in cell.get('outputs', []):
        if o.get('name') == 'stdout': o['text'] = _re_ansi_escape.sub('', o.text)

In [21]:
def _get_cell_id(id_length=36): return uuid.uuid4().hex[:id_length]

def _get_md_cell(content="<!--- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT!-->"):
    return AttrDict({'cell_type': 'markdown', 'id': f'{_get_cell_id()}',
                     'metadata': {}, 'source': f'{content}'})

def insert_warning(nb):
    "Insert Autogenerated Warning Into Notebook after the first cell."
    nb.cells = nb.cells[:1] + [_get_md_cell()] + nb.cells[1:]

In [22]:
procs = [rm_blank_proc, strip_ansi_proc]
post_procs = [insert_warning]

In [23]:
def exp_md(path, procs=None, post_procs=None):
    nbp = NBProcessor(path, procs)
    nb = nbp.nb
    lang = nb.metadata.language_info.name
    nbp.process()
    for proc in L(post_procs): proc(nb)
    return nbp

In [24]:
path = '../tests/docs_test.ipynb'
nbp = exp_md(path, procs, post_procs)
nb = nbp.nb

In [25]:
nb.cells[:3]

[## a title,
 - cell_type: markdown
 - id: ef72d4f5ad4f41e09854310870685fb2
 - metadata: 
 
 - source: <!--- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT!-->,
 some md]

In [36]:
print(exp_md('../tests/docs_test.ipynb', procs))

## a title
some md

```python
1+1
```

```
  2
```

```python
%%html
<b>a test</b>
```

<b>a test</b>

```python
import pandas as pd
print('hi')
pd.DataFrame(dict(a=[1,2]))
```

<div>

<style scoped>

    .dataframe tbody tr th:only-of-type {

        vertical-align: middle;

    }



    .dataframe tbody tr th {

        vertical-align: top;

    }



    .dataframe thead th {

        text-align: right;

    }

</style>

<table border="1" class="dataframe">

  <thead>

    <tr style="text-align: right;">

      <th></th>

      <th>a</th>

    </tr>

  </thead>

  <tbody>

    <tr>

      <th>0</th>

      <td>1</td>

    </tr>

    <tr>

      <th>1</th>

      <td>2</td>

    </tr>

  </tbody>

</table>

</div>


In [26]:
from nbconvert.preprocessors import Preprocessor
from nbconvert import MarkdownExporter
from nbconvert.preprocessors import TagRemovePreprocessor
from nbdev.imports import get_config
from traitlets.config import Config
from pathlib import Path
import re, uuid

In [27]:
__file__ = '../nbprocess/export.py'

In [28]:
def get_mdx_exporter(template_file='ob.tpl'):
    """A mdx notebook exporter which composes many pre-processors together."""
    c = Config()
#     c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell", "hide")
#     c.TagRemovePreprocessor.remove_all_outputs_tags = ("remove_output", "remove_outputs", "hide_output", "hide_outputs")
#     c.TagRemovePreprocessor.remove_input_tags = ('remove_input', 'remove_inputs', "hide_input", "hide_inputs")
#     pp = [InjectMeta, WriteTitle, CleanMagics, BashIdentify, MetaflowTruncate,
#           MetaflowSelectSteps, UpdateTags, InsertWarning, TagRemovePreprocessor, CleanFlags, CleanShowDoc, RmEmptyCode,
#           StripAnsi, HideInputLines, Black, ImageSave, ImagePath, HTMLEscape]
#     c.MarkdownExporter.preprocessors = pp
    tpl_dir = Path(__file__).parent/'templates/'
    tpl_file = tpl_dir/f"{template_file}"
    if not tpl_file.exists(): raise ValueError(f"{tpl_file} does not exist in {tpl_dir}")
    c.MarkdownExporter.template_file = str(tpl_file)
    return MarkdownExporter(config=c)

In [33]:
nb = read_nb('../tests/minimal.ipynb')

In [40]:
js = Path('../tests/minimal.ipynb').read_json()
nb = dict2obj(js)

In [44]:
nb.cells

(#3) [- cell_type: markdown
- metadata: 

- source: 
  - # A minimal notebook,- cell_type: code
- execution_count: 1
- metadata: 

- outputs: 
  - 
    - data: 
      - text/plain: 
        - 2
    - execution_count: 1
    - metadata: 

    - output_type: execute_result
- source: 
  - # Do some arithmetic

  - 1+1,- cell_type: code
- execution_count: None
- metadata: 

- outputs: 

- source:]

In [41]:
exp = get_mdx_exporter()

In [42]:
exp.from_notebook_node(nb)[0]

NotebookValidationError: [- cell_type: markdown
- metadata: 

- source: 
  - # A minimal notebook, - cell_type: code
- execution_count: 1
- metadata: 

- outputs: 
  - 
    - data: 
      - text/plain: 
        - 2
    - execution_count: 1
    - metadata: 

    - output_type: execute_result
- source: 
  - # Do some arithmetic

  - 1+1, - cell_type: code
- execution_count: None
- metadata: 

- outputs: 

- source:] is not of type 'array'

Failed validating 'type' in notebook['properties']['cells']:

On instance['cells']:
[- cell_type: markdown
- metadata: 

- source: 
  - # A minimal notebook, - cell_type: code
- execution_count: 1
- metadata: 

- outputs: 
  - 
    - data: 
      - text/plain: 
        - 2
    - execution_count: 1
    - metadata: 

    - output_type: execute_result
- source: 
  - # Do some arithmetic

  - 1+1, - cell_type: code
- execution_count: None
- metadata: 

- outputs: 

- source:]